
# Primeros pasos con modelos para datos meteorológicos de AEMET

En esta notebook vamos a cargar los datos limpios diarios, mensuales y anuales que tienes, y hacer una prueba sencilla con tres modelos diferentes:  
- Árbol de decisión para clasificar si hubo lluvia o no  
- Regresión lineal para predecir la cantidad de lluvia  
- Red LSTM para predecir temperatura media como serie temporal  

Vamos paso a paso, con explicaciones sencillas para que entiendas bien cada parte.

---

**Instrucciones previas:**  
Para ejecutar esta notebook necesitas tener instalado Jupyter Notebook o JupyterLab.  
Si no lo tienes, abre tu consola (cmd o PowerShell) y ejecuta:

```
pip install notebook
```

Luego, para lanzar Jupyter, navega en la consola a la carpeta donde está esta notebook y escribe:

```
jupyter notebook
```

Esto abrirá una ventana en tu navegador con la lista de archivos. Haz clic en esta notebook para abrirla y ejecutar las celdas.

---

¡Empecemos!


In [1]:

# Celda 1 - Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:

# Celda 2 - Cargar datos limpios (ajusta las rutas a tus archivos)
df_diarios = pd.read_csv('src/data/limpios/alcantarilla_diarios.csv')
df_mensuales = pd.read_csv('src/data/limpios/mensuales.csv')
df_anuales = pd.read_csv('src/data/limpios/anuales.csv')

print("Datos diarios cargados:", df_diarios.shape)
print("Datos mensuales cargados:", df_mensuales.shape)
print("Datos anuales cargados:", df_anuales.shape)


In [ ]:

# Celda 3 - Exploración rápida de los datos diarios
print(df_diarios.head())
print(df_diarios.info())


In [ ]:

# Celda 4 - Preparar datos para un modelo simple:  
# Queremos predecir si llovió o no (variable 'prec' > 0)

# Crear columna 'lluvia' binaria: 1 si lluvia > 0, 0 si no
df_diarios['lluvia'] = (df_diarios['prec'] > 0).astype(int)

# Elegimos características (features) numéricas para entrenar
features = ['tmed', 'tmin', 'tmax', 'velmedia', 'racha', 'presMax', 'presMin', 'hrMedia', 'sol']
X = df_diarios[features].fillna(0)  # Rellenar nulos con 0 para simplificar
y = df_diarios['lluvia']

print("Características seleccionadas:", X.columns)


In [ ]:

# Celda 5 - Dividir los datos en entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Tamaño de entrenamiento:", X_train.shape)
print("Tamaño de prueba:", X_test.shape)


In [ ]:

# Celda 6 - Entrenar un Árbol de Decisión para clasificación de lluvia

clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

score_train = clf.score(X_train, y_train)
score_test = clf.score(X_test, y_test)

print(f"Precisión en entrenamiento: {score_train:.2f}")
print(f"Precisión en prueba: {score_test:.2f}")


In [ ]:

# Celda 7 - Entrenar un modelo de regresión para predecir la cantidad de lluvia (prec)

# Usamos solo días con lluvia para este modelo
precip_dias = df_diarios[df_diarios['prec'] > 0]
X_prec = precip_dias[features].fillna(0)
y_prec = precip_dias['prec']

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_prec, y_prec, test_size=0.2, random_state=42)

reg = LinearRegression()
reg.fit(X_train_reg, y_train_reg)

score_reg_train = reg.score(X_train_reg, y_train_reg)
score_reg_test = reg.score(X_test_reg, y_test_reg)

print(f"R2 en entrenamiento (precipitación): {score_reg_train:.2f}")
print(f"R2 en prueba (precipitación): {score_reg_test:.2f}")


In [ ]:

# Celda 8 - Preparar datos para LSTM (serie temporal)

# Vamos a usar la temperatura media diaria para predecir la siguiente

# Ordenar por fecha
df_diarios = df_diarios.sort_values('fecha')

# Seleccionar la columna para la serie temporal
serie = df_diarios['tmed'].fillna(method='ffill').values  # Rellenar con el último valor válido

# Normalizar (escalar) la serie entre 0 y 1 para que LSTM funcione mejor
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
serie_scaled = scaler.fit_transform(serie.reshape(-1,1))

# Función para crear secuencias (ventanas deslizantes)
def crear_secuencias(data, pasos=5):
    X, y = [], []
    for i in range(len(data) - pasos):
        X.append(data[i:i+pasos])
        y.append(data[i+pasos])
    return np.array(X), np.array(y)

pasos = 5
X_lstm, y_lstm = crear_secuencias(serie_scaled, pasos)

# Dividir en entrenamiento y prueba
split = int(len(X_lstm) * 0.8)
X_train_lstm, X_test_lstm = X_lstm[:split], X_lstm[split:]
y_train_lstm, y_test_lstm = y_lstm[:split], y_lstm[split:]

print("Formas de los datos LSTM:", X_train_lstm.shape, y_train_lstm.shape)


In [ ]:

# Celda 9 - Crear y entrenar modelo LSTM

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(pasos, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Entrenar con EarlyStopping para no sobreentrenar
early_stop = EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(X_train_lstm, y_train_lstm,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_test_lstm, y_test_lstm),
                    callbacks=[early_stop],
                    verbose=1)


In [ ]:

# Celda 10 - Visualizar la evolución del error en entrenamiento y prueba

import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Error entrenamiento')
plt.plot(history.history['val_loss'], label='Error validación')
plt.legend()
plt.show()
